# Neural Network

Trying to solve a prediction problem using sci-kit learn Neural network models. 


## Data Loading 

Everything was already processed so, just using that.

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from sklearn import svm
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt 
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
os.getcwd()
#os.chdir('c:\\Users\\amcfa\\gitfiles\\Projects\\MastersWork\\shorter_ML_Projects\\Machine-Learning-_-Prediction')
X1=pd.read_csv(os.getcwd()+ '/X1.csv')
X2=pd.read_csv(os.getcwd()+ '/X2.csv')
from sklearn.utils import shuffle
X1 = shuffle(X1)
X2=shuffle(X2)
y_train=X1['Data_x'].reset_index(drop=True) # Separating all of the data
y_test=X2['Data_x'].reset_index(drop=True)
X_train1 = X1.iloc[:,3: ]
X_test1 = X2.iloc[:,3: ]

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
scaler = StandardScaler()
scaler.fit(X_train1)
X_train=scaler.transform(X_train1)
X_test=scaler.transform(X_test1) 

## A first Pass
Just using the model as it comes. No parameter optimization here.

In [ ]:
regr = MLPRegressor(max_iter=500).fit(X_train, y_train)
regr.predict(X_test)
regr.score(X_test, y_test)

In [ ]:
regr = MLPRegressor().fit(X_train, y_train)
y_pred=regr.predict(X_test)


In [ ]:
y_true=y_test

In [ ]:
results_trial1=pd.DataFrame(y_pred,y_true).reset_index()

In [ ]:
results_trial1=pd.DataFrame(y_pred,y_true).reset_index()
results_trial1=results_trial1.rename(columns={'Data_x':'true',0:'predicted'})
results_trial1['how_off']= ((results_trial1['true']-results_trial1['predicted'])/results_trial1['true'])*100
results_trial1.head

In [ ]:
r1 = np.arange(len(results_trial1['true']))

plt.bar(r1, results_trial1['true'], color='#7f6d5f', edgecolor='white', label='true values')
plt.scatter(r1, results_trial1['predicted'], color='#557f2d', edgecolor='white', label='predicted values')
plt.xlabel('Run #')
plt.ylabel('Predicted Values')
plt.title('Predicted value vs true value')

plt.legend()

In [ ]:
print('Mean absolute Error',mean_absolute_error(y_true, y_pred))
print('Mean squared error', mean_squared_error(y_true, y_pred))
print(('Explained variance score',explained_variance_score(y_true, y_pred)))
print(('R_2 score',r2_score(y_true, y_pred)))

The model needed some optimization for sure. 


# Optimization


### Selecting activation function

Default activation is `relu` \
references*
https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

Will try:\
`identity`-no-op activation, useful to implement linear bottleneck, returns f(x) = x\
`logistic`-The logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).\
`tahn`-the hyperbolic tan function, returns f(x) = tanh(x).

In [ ]:
m=['relu','identity','logistic','tanh']
abc=[]
for i,j in (enumerate(m)):
    mod=j
    regr=MLPRegressor(activation=j).fit(X_train, y_train)
    regr.predict(X_test)
    y_pred=regr.predict(X_test)
    score=regr.score(X_test, y_test)
    abc.append([mod,score,mean_absolute_error(y_true, y_pred),mean_squared_error(y_true, y_pred),
                explained_variance_score(y_true, y_pred),r2_score(y_true, y_pred)])

In [ ]:
scores=pd.DataFrame(abc,columns=['iter','score','MAE','MSE','EV','R2'])
scores

looks like identity is the winner here. 


## Changing iterations

In [ ]:
m=2500
abc=[]
defg=[]
#This needs to run 3 times and in those 3 times. Needs to give results
for i in range(1,m,100):
    regr=MLPRegressor(activation='identity',max_iter=i).fit(X_train, y_train)
    regr.predict(X_test)
    y_pred=regr.predict(X_test)
    score=regr.score(X_test, y_test)
    abc.append([i,score,mean_absolute_error(y_true, y_pred)
                ,mean_squared_error(y_true, y_pred)
                ,explained_variance_score(y_true, y_pred),r2_score(y_true, y_pred)])

In [ ]:
scores=pd.DataFrame(abc,columns=['iter','score','MAE','MSE','EV','R2'])

In [ ]:
plt.plot(scores['MAE'])

In [ ]:
plt.plot(scores['MSE'])

In [ ]:
plt.plot(scores['EV'])

Best iterations is 2300

### Changing the solver

Going to change the solver now- the only alternative to adam is `lbfgs`

In [ ]:
m=['lbfgs']
abc=[]
for i,j in (enumerate(m)):
    mod=j
    regr=MLPRegressor(activation='identity',max_iter=2300,solver=j).fit(X_train, y_train)
    regr.predict(X_test)
    y_pred=regr.predict(X_test)
    score=regr.score(X_test, y_test)
    abc.append([mod,score,mean_absolute_error(y_true, y_pred),mean_squared_error(y_true, y_pred),
                explained_variance_score(y_true, y_pred),r2_score(y_true, y_pred)])

In [ ]:
scores=pd.DataFrame(abc,columns=['solver','score','MAE','MSE','EV','R2'])

In [ ]:
results_trial1=pd.DataFrame(y_pred,y_true).reset_index()

In [ ]:
results_trial1=pd.DataFrame(y_pred,y_true).reset_index()
results_trial1=results_trial1.rename(columns={'Data_x':'true',0:'predicted'})
results_trial1['how_off']= ((results_trial1['true']-results_trial1['predicted'])/results_trial1['true'])*100
results_trial1.head

In [ ]:
r1 = np.arange(len(results_trial1['true']))

plt.bar(r1, results_trial1['true'], color='#7f6d5f', edgecolor='white', label='true values')
plt.scatter(r1, results_trial1['predicted'], color='#557f2d', edgecolor='white', label='predicted values')
plt.xlabel('Run #')
plt.ylabel('Predicted Values')
plt.title('Predicted value vs true value')

plt.legend()

In [ ]:
from sklearn.metrics import d2_absolute_error_score
d2_absolute_error_score(y_true, y_pred,multioutput='raw_values')

In [ ]:
print('Mean absolute Error',mean_absolute_error(y_true, y_pred))
print('Mean squared error', mean_squared_error(y_true, y_pred))
print(('Explained variance score',explained_variance_score(y_true, y_pred)))
print(('R_2 score',r2_score(y_true, y_pred)))

# Results

The Neural Network Model performed very well. With a prediction accuracy of 100%.\
The `MAE` was 0.032168487841461764\
The `MSE` was 0.0030038938705200698\
The `Explained variance` was 0.9999999998453984\
The $ R^2 $ was 0.9999999998356973